In [0]:
import torch
import torchvision
from torchvision import transforms, datasets

In [0]:
#torchivision has premade datasets which I have used her

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True)

test = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=True)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
#visualiszing data in our training set
for data in train:
    print("input image shape: ", data[0].shape)
    print("labels shape: ", data[1].shape)
    break

input image shape:  torch.Size([10, 3, 32, 32])
labels shape:  torch.Size([10])


In [0]:
"""
verifying the training dataset is balanced and not skewed, 
here the keys of the dictionary represent the classes
"""

total = 0
counter_dict = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0}

for data in train:
    Xs, ys = data
    for y in ys:
        counter_dict[int(y)] += 1
        total += 1

print(counter_dict)

{0: 5000, 1: 5000, 2: 5000, 3: 5000, 4: 5000, 5: 5000, 6: 5000, 7: 5000, 8: 5000, 9: 5000}


In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5, padding = (2,2))
        self.conv3 = nn.Conv2d(16, 32, 3, padding = (1,1))
        self.fc1 = nn.Linear(32 * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ConvNet()

In [0]:
import torch.optim as optim

criterion =nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.0001)

epochs = 10

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for j, data in enumerate(train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        #creating a one-hot vector representation of labels
        label = torch.zeros(10,10)     
        for i in range(10):
            label[i,labels[i]] = 1

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if j % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, j + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.080
[1,  4000] loss: 0.072
[2,  2000] loss: 0.067
[2,  4000] loss: 0.065
[3,  2000] loss: 0.062
[3,  4000] loss: 0.060
[4,  2000] loss: 0.058
[4,  4000] loss: 0.057
[5,  2000] loss: 0.054
[5,  4000] loss: 0.054
[6,  2000] loss: 0.052
[6,  4000] loss: 0.051
[7,  2000] loss: 0.050
[7,  4000] loss: 0.050
[8,  2000] loss: 0.047
[8,  4000] loss: 0.048
[9,  2000] loss: 0.045
[9,  4000] loss: 0.046
[10,  2000] loss: 0.044
[10,  4000] loss: 0.044
Finished Training


In [0]:
print(outputs.shape, labels.shape, label.shape)

torch.Size([10, 10]) torch.Size([10]) torch.Size([10, 10])


In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in test:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 64 %
